In [20]:
type BoxStm = Box<Stm>;
type BoxExp = Box<Exp>;
type BoxExpList = Box<ExpList>;

#[derive(Debug)]
enum Binop {
    Plus,
    Minus,
    Times,
    Div,
}

#[derive(Debug)]
enum Stm {
    Compound(BoxStm, BoxStm),
    Assign { id: String, exp: BoxExp },
    Print(BoxExpList),
}

impl Stm {
    fn new_box_compound(stm1: BoxStm, stm2: BoxStm) -> BoxStm {
        Box::new(Self::Compound(stm1, stm2))
    }

    fn new_box_assign(id: String, exp: BoxExp) -> BoxStm {
        Box::new(Self::Assign { id, exp })
    }

    fn new_box_print(exp_list: BoxExpList) -> BoxStm {
        Box::new(Self::Print(exp_list))
    }
}

#[derive(Debug)]
enum Exp {
    Id(String),
    Num(isize),
    Op {
        left: BoxExp,
        oper: Binop,
        right: BoxExp,
    },
    Eseq {
        stm: BoxStm,
        exp: BoxExp,
    },
}

impl Exp {
    fn new_box_id(id: String) -> BoxExp {
        Box::new(Self::Id(id))
    }

    fn new_box_num(num: isize) -> BoxExp {
        Box::new(Self::Num(num))
    }

    fn new_box_op(left: BoxExp, oper: Binop, right: BoxExp) -> BoxExp {
        Box::new(Self::Op { left, oper, right })
    }

    fn new_box_eseq(stm: BoxStm, exp: BoxExp) -> BoxExp {
        Box::new(Exp::Eseq { stm, exp })
    }
}

#[derive(Debug)]
enum ExpList {
    Pair { head: BoxExp, tail: BoxExpList },
    Last(BoxExp),
}

impl ExpList {
    fn new_box_pair(head: BoxExp, tail: BoxExpList) -> BoxExpList {
        Box::new(ExpList::Pair { head, tail })
    }

    fn new_box_last(last: BoxExp) -> BoxExpList {
        Box::new(ExpList::Last(last))
    }
}

// a := 5 + 3 ; b := ( print ( a , a - 1) , 10 * a) ; print ( b )
let prog = Stm::new_box_compound(
    Stm::new_box_assign(
        "a".to_owned(),
        Exp::new_box_op(Exp::new_box_num(5), Binop::Plus, Exp::new_box_num(3)),
    ),
    Stm::new_box_compound(
        Stm::new_box_assign(
            "b".to_owned(),
            Exp::new_box_eseq(
                Stm::new_box_print(ExpList::new_box_pair(
                    Exp::new_box_id("a".to_owned()),
                    ExpList::new_box_last(Exp::new_box_op(
                        Exp::new_box_id("a".to_owned()),
                        Binop::Minus,
                        Exp::new_box_num(1),
                    )),
                )),
                Exp::new_box_op(
                    Exp::new_box_num(10),
                    Binop::Times,
                    Exp::new_box_id("a".to_owned()),
                ),
            ),
        ),
        Stm::new_box_print(ExpList::new_box_last(Exp::new_box_id("b".to_owned()))),
    ),
);


（1） 写一个函数 int maxargs(A_stm)，告知给定语句中任意子表达式内的 print 语句的参数个数。例如，maxargs(prog) 的值是 2。

In [21]:
fn maxargs(stms: &BoxStm) -> usize {
    fn exp_maxargs(exp: &BoxExp) -> usize {
        match &**exp {
            Exp::Eseq { stm, exp } => maxargs(stm) + exp_maxargs(exp),
            Exp::Op { left, right, .. } => exp_maxargs(left).max(exp_maxargs(right)),
            _ => 0,
        }
    }

    fn explist_maxargs(exp_list: &BoxExpList) -> usize {
        match &**exp_list {
            ExpList::Pair { head, tail } => exp_maxargs(head).max(1 + explist_maxargs(tail)),
            ExpList::Last(exp) => 1 + exp_maxargs(exp),
        }
    }

    match &**stms {
        Stm::Compound(stm1, stm2) => maxargs(stm1).max(maxargs(stm2)),
        Stm::Assign { exp, .. } => exp_maxargs(exp),
        Stm::Print(exp_list) => explist_maxargs(exp_list),
    }
}

maxargs(&prog)

2

（2）写一个函数 void interp(A_stm), 对一个用这种直线式程序语言写的程序进行“解释”，为了用“函数式程序设计”风格来编写该函数（这种风格不使用赋值语句），要在声明局部变量的同时对它进行初始化。

In [22]:
type OptBoxTable = Option<Box<Table>>;

struct Table {
    id: String,
    value: isize,
    tail: OptBoxTable,
}

impl Table {
    fn update(table: OptBoxTable, id: String, value: isize) -> OptBoxTable {
        Some(Box::new(Table {
            id,
            value,
            tail: table,
        }))
    }

    fn lookup(table: Option<&Box<Table>>, id: &str) -> Option<isize> {
        if let Some(table) = table {
            if table.id == id {
                return Some(table.value);
            }
            return Self::lookup(table.tail.as_ref(), id);
        }
        None
    }
}

type IntAndTable = (isize, OptBoxTable);

fn interp_exp(exp: BoxExp, table: OptBoxTable) -> IntAndTable {
    match *exp {
        Exp::Id(id) => (
            Table::lookup(table.as_ref(), &id)
                .unwrap_or_else(|| panic!("error: '{}' is not defined", id)),
            table,
        ),
        Exp::Num(value) => (value, table),
        Exp::Op { left, oper, right } => {
            let (left_value, table) = interp_exp(left, table);
            let (right_value, table) = interp_exp(right, table);
            let value = match oper {
                Binop::Plus => left_value + right_value,
                Binop::Minus => left_value - right_value,
                Binop::Times => left_value * right_value,
                Binop::Div => left_value / right_value,
            };
            (value, table)
        }
        Exp::Eseq { stm, exp } => interp_exp(exp, interp_stm(stm, table)),
    }
}

fn print_exp_list(exp_list: BoxExpList, table: OptBoxTable) -> OptBoxTable {
    match *exp_list {
        ExpList::Pair { head, tail } => {
            let (value, table) = interp_exp(head, table);
            print!("{} ", value);
            print_exp_list(tail, table)
        }
        ExpList::Last(exp) => {
            let (value, table) = interp_exp(exp, table);
            println!("{}", value);
            table
        }
    }
}

fn interp_stm(stm: BoxStm, table: OptBoxTable) -> OptBoxTable {
    match *stm {
        Stm::Compound(stm1, stm2) => interp_stm(stm2, interp_stm(stm1, table)),
        Stm::Assign { id, exp } => {
            let (value, table) = interp_exp(exp, table);
            Table::update(table, id, value)
        }
        Stm::Print(exp_list) => print_exp_list(exp_list, table),
    }
}

interp_stm(prog, None);

8 7
80
